In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

#result = '2020_01_24_01_11_39'
#result = '2020_01_25_05_00_12'
#result = '2020_01_25_06_58_19'
result = 'latest'

#base_dir = f'salomon/{result}/'
base_dir = f'run/outputs/{result}/'

base_dir = os.getenv('B_RESULT_DIR', base_dir)
data = pd.read_csv(os.path.join(base_dir, "stats.csv"), names=['stat', 'rank', 'size', 'round', 'time_ms'])

In [ ]:
display(HTML('<h1>' + os.path.basename(os.path.realpath(base_dir)) + '</h1>'))
with open(os.path.join(base_dir, 'info')) as f:
    display(HTML(f'<pre>{f.read()}</pre>'))
try:
    with open(os.path.join(base_dir, 'env')) as f:
        display(HTML(f'<pre>{f.read()}</pre>'))
except FileNotFoundError:
    pass

In [ ]:
with open(os.path.join(base_dir, 'out')) as f:
    display(HTML(f'<pre>{f.read()}</pre>'))

In [ ]:
with open(os.path.join(base_dir, 'checksums')) as f:
    display(HTML(f'<pre>{f.read()}</pre>'))

In [ ]:
from IPython.display import Image
for path in glob.glob(os.path.join(base_dir, "**/*.thumbnail.png"), recursive=True):
    print(path)
    display(Image(filename=path))

In [ ]:
for stat in data['stat'].unique():
    display(HTML(f"<h1>{stat}</h1>"))
    overview = data[data['stat'] == stat].pivot_table(index=['size', 'round'], columns='rank', values='time_ms')
    display(overview)
    
    mean_per_node = data[data['stat'] == stat].pivot_table(index=['size'], columns='rank', values='time_ms')
    display(mean_per_node)
    mean_per_node.T.mean().plot.bar()
    plt.show()
    
    global_mean = mean_per_node.T.mean()
    if 1 in global_mean:
        global_mean = global_mean[1] / global_mean
        plt.plot(global_mean, linestyle='none', marker='o')
        x = np.linspace(1, len(global_mean))
        plt.plot(x, x)
        plt.show()

In [ ]:
convolution = data[data['stat'] == 'convolution'].pivot_table(index=['size'], columns='rank', values='time_ms')
gather = data[data['stat'] == 'gather'].pivot_table(index=['size'], columns='rank', values='time_ms')
total = (convolution + gather).T.mean()

display(total)

speedup = total[1] / total
plt.plot(speedup, linestyle='none', marker='o')
x = np.linspace(1, len(speedup))
plt.plot(x, x)
plt.show()